# Predicting Heart Failure

In [15]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [16]:
set.seed(1)

In [29]:
heart_failure <- read_csv("heart_failure_clinical_records.csv") %>% as_tibble()
heart_failure

Parsed with column specification:
cols(
  age = col_double(),
  anaemia = col_double(),
  creatinine_phosphokinase = col_double(),
  diabetes = col_double(),
  ejection_fraction = col_double(),
  high_blood_pressure = col_double(),
  platelets = col_double(),
  serum_creatinine = col_double(),
  serum_sodium = col_double(),
  sex = col_double(),
  smoking = col_double(),
  time = col_double(),
  DEATH_EVENT = col_double()
)



age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
75,0,582,0,20,1,265000,1.9,130,1,0,4,1
55,0,7861,0,38,0,263358,1.1,136,1,0,6,1
65,0,146,0,20,0,162000,1.3,129,1,1,7,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
45,0,2060,1,60,0,742000,0.8,138,0,0,278,0
45,0,2413,0,38,0,140000,1.4,140,1,1,280,0
50,0,196,0,45,0,395000,1.6,136,1,1,285,0


In [30]:
heart_failure_split <- initial_split(heart_failure, prop = 0.75, strata = DEATH_EVENT)
heart_failure_train <- training(heart_failure_split)
heart_failure_test <- testing(heart_failure_split)

In [32]:
glimpse(heart_failure_train)

Rows: 225
Columns: 13
$ age                      <dbl> 75, 65, 50, 65, 90, 75, 60, 65, 80, 75, 62, …
$ anaemia                  <dbl> 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,…
$ creatinine_phosphokinase <dbl> 582, 146, 111, 160, 47, 246, 315, 157, 123, …
$ diabetes                 <dbl> 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,…
$ ejection_fraction        <dbl> 20, 20, 20, 20, 40, 15, 60, 65, 35, 38, 25, …
$ high_blood_pressure      <dbl> 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,…
$ platelets                <dbl> 265000, 162000, 210000, 327000, 204000, 1270…
$ serum_creatinine         <dbl> 1.9, 1.3, 1.9, 2.7, 2.1, 1.2, 1.1, 1.5, 9.4,…
$ serum_sodium             <dbl> 130, 129, 137, 116, 132, 137, 131, 138, 133,…
$ sex                      <dbl> 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,…
$ smoking                  <dbl> 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,…
$ time                     <dbl> 4, 7, 7, 8, 8, 10, 10, 10, 10, 10, 10, 11, 1…
$ DEATH_EVENT              <db